## Face Recognition – Unlock Your Computer With Your Face!


### Step 1 - Create Training Data

In [89]:
import cv2
import numpy as np

# Load HAAR face classifier
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Load functions
def face_extractor(img):
    # Function detects faces and returns the cropped face
    # If no face detected, it returns the input image
    
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    
    if faces is ():
        return None
    
    # Crop all faces found
    for (x,y,w,h) in faces:
        cropped_face = img[y:y+h, x:x+w]

    return cropped_face

# Initialize Webcam
cap = cv2.VideoCapture(0)
count = 0

# Collect 100 samples of your face from webcam input
while True:

    ret, frame = cap.read()
    if face_extractor(frame) is not None:
        count += 1
        face = cv2.resize(face_extractor(frame), (200, 200))
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        # Save file in specified directory with unique name
        file_name_path = './faces/user/' + str(count) + '.jpg'
        cv2.imwrite(file_name_path, face)

        # Put count on images and display live count
        cv2.putText(face, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        cv2.imshow('Face Cropper', face)
        
    else:
        print("Face not found")
        pass

    if cv2.waitKey(1) == 13 or count == 100: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()      
print("Collecting Samples Complete")

<>:15: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:15: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-89-629aa437f198>:15: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():


### Step 2 - Train Model

In [90]:
cap.release()

In [91]:
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

# Get the training data we previously made
data_path = './faces/user/'
onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path, f))]

# Create arrays for training data and labels
Training_Data, Labels = [], []

# Open training images in our datapath
# Create a numpy array for training data
for i, files in enumerate(onlyfiles):
    image_path = data_path + onlyfiles[i]
    images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    Training_Data.append(np.asarray(images, dtype=np.uint8))
    Labels.append(i)

# Create a numpy array for both training data and labels
Labels = np.asarray(Labels, dtype=np.int32)

# Initialize facial recognizer
# model = cv2.face.createLBPHFaceRecognizer()
# NOTE: For OpenCV 3.0 use cv2.face.createLBPHFaceRecognizer()
# pip install opencv-contrib-python
# model = cv2.createLBPHFaceRecognizer()

ajmal_model  = cv2.face_LBPHFaceRecognizer.create()
# Let's train our model 
ajmal_model.train(np.asarray(Training_Data), np.asarray(Labels))
print("Model trained sucessefully")


Model trained sucessefully


### Step 3 - Run Our Facial Recognition

In [92]:
import os
import cv2
import subprocess
import threading
import numpy as np
import sendmail as sd


face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def face_detector(img, size=0.5):
    
    # Convert image to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        return img, []
    
    
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        roi = img[y:y+h, x:x+w]
        roi = cv2.resize(roi, (200, 200))
    return img, roi


# Open Webcam
cap = cv2.VideoCapture(0)

email_sent = False
while True:

    ret, frame = cap.read()
    
    image, face = face_detector(frame)
    
    try:
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        # Pass face to prediction model
        # "results" comprises of a tuple containing the label and the confidence value
        results = ajmal_model.predict(face)
        # harry_model.predict(face)
        
        if results[1] < 500:
            confidence = int( 100 * (1 - (results[1])/400) )
            display_string = str(confidence) + '% Confident it is User'
            
        cv2.putText(image, display_string, (100, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (255,120,150), 2)
        
        if confidence > 90:
            cv2.putText(image, "Hey Ajmal", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            cv2.imshow('Face Recognition', image)
         
            if not email_sent:
                print("Sending Email")
                email_sent = True
                file_name_path = "./faces/email/" + "ajmal" + ".jpg"
                cv2.imwrite(file_name_path, face)
                t1 = threading.Thread(target=sd.sendmail)
                t1.start()
                print("Launching EC2 Instnace")
                ec2_launching = "aws ec2  run-instances --image-id ami-0ad704c126371a549 --count 1 --instance-type t2.micro --key-name summer"
                subprocess.call(ec2_launching, shell=True)
                print("Getting the Instance id of Currently launched EC2")
                instance_id = subprocess.getoutput('aws ec2 describe-instances  --query "sort_by(Reservations[].Instances[], &LaunchTime)[-1].[InstanceId]" --output text')
                print(instance_id)
                print("Getting the avaialbility zone of currently launched EC2 instance")
                instance_availability_zone = subprocess.getoutput('aws ec2 describe-instances --query "sort_by(Reservations[].Instances[],  &LaunchTime)[-1].[Placement.AvailabilityZone]" --output text')
                print(instance_availability_zone)
                print("Creating EBS of size 5GB")
                subprocess.call("aws ec2 create-volume  --availability-zone {0} --size 5".format(instance_availability_zone), shell=True)
                print("Getting the Volume Id of EBS")
                volume_id = subprocess.getoutput('aws ec2 describe-volumes  --query "Volumes[-1].[VolumeId]" --output text')
                print(volume_id)
                print("Attaching the newly created EBS Volume with previously launched EC2 Instnace")
                subprocess.call('aws ec2 attach-volume --device /dev/xvdj --instance-id {0} --volume-id {1}'.format(instance_id, id_volume), shell=True)
                print("Successfully Attached EBS Volume with EC2 Instance")
                
        else:
            
            cv2.putText(image, "I dont know, whow are you", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
            cv2.imshow('Face Recognition', image )

    except:
        cv2.putText(image, "No Face Found", (220, 120) , cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.putText(image, "looking for face", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.imshow('Face Recognition', image )
        pass
        
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()     

<>:16: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:16: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-92-6ab9ea1d4ded>:16: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():


Sending Email
Launching EC2 Instnace
Getting the Instance id of Currently launched EC2
i-01d052441a738154d
Getting the avaialbility zone of currently launched EC2 instance
ap-south-1a
Creating EBS of size 5GB
Getting the Volume Id of EBS
vol-0458378785ca8ba43
Attaching the newly created EBS Volume with previously launched EC2 Instnace
Successfully Attached EBS Volume with EC2 Instance


In [3]:
import subprocess

In [71]:
instance_id = subprocess.getoutput('aws ec2 describe-instances  --query "sort_by(Reservations[].Instances[], &LaunchTime)[-1].[InstanceId]" --output text')


In [72]:
print(instance_id)

i-0428790c2d273dbeb


In [73]:
instance_availability_zone = subprocess.getoutput('aws ec2 describe-instances --query "sort_by(Reservations[].Instances[],  &LaunchTime)[-1].[Placement.AvailabilityZone]" --output text')

In [74]:
print(instance_availability_zone)

ap-south-1a


In [34]:
volume_launching = subprocess.call("aws ec2 create-volume  --availability-zone {0} --size 5".format(instance_availability_zone), shell=True)

In [35]:
print(volume_launching)

0


In [82]:
volume_id = subprocess.getoutput('aws ec2 describe-volumes  --query "Volumes[-1].[VolumeId]" --output text')

In [84]:
print(volume_id)

vol-0b055cbbaf4a12649


In [85]:
attaching_volume = subprocess.call('aws ec2 attach-volume --device /dev/xvdy --instance-id {0} --volume-id {1}'.format(instance_id, id_volume), shell=True)

In [88]:
print(attaching_volume)

"'i-0428790c2d273dbeb' is not recognized as an internal or external command,\noperable program or batch file."